In [5]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = "better-rag-index"

In [6]:
from langchain_openai import OpenAIEmbeddings

emb = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key=OPENAI_API_KEY
)

In [27]:
import os
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings

# =========================
# 환경 변수
# =========================
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
INDEX_NAME = "realestate"

# =========================
# Pinecone 연결
# =========================
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)

# =========================
# 임베딩 모델
# =========================
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# =========================
# Pinecone 후보 검색
# =========================
def retrieve_candidates(query: str, top_k: int = 50):
    qvec = embeddings.embed_query(query)
    res = index.query(
        vector=qvec,
        top_k=top_k,
        include_metadata=True
    )
    return res["matches"]

# =========================
# 점수 보정
# =========================
def adjusted_score(match):
    score = match["score"]
    md = match["metadata"]

    priority = md.get("priority", 99)
    text = md.get("text", "")
    article = md.get("article", "")

    # 판례 가중치
    if priority == 50:
        score += 0.4

    # 대항력 핵심 키워드
    for term in ["대항력", "우선변제권", "확정일자"]:
        if term in text:
            score += 0.08

    # 제3조 계열 추가 가중치
    if article.startswith("제3조"):
        score += 0.3

    return score

# =========================
# 최종 Retrieval (안정화)
# =========================
def retrieve_final(
    query: str,
    top_k: int = 6,
    min_law: int = 2,
    min_case: int = 1,
):
    matches = retrieve_candidates(query)

    # 점수 보정 후 정렬
    ranked = sorted(matches, key=lambda m: -adjusted_score(m))

    laws = []
    cases = []
    others = []

    for m in ranked:
        md = m["metadata"]
        priority = md.get("priority", 99)

        if priority == 1:
            laws.append(m)
        elif priority == 50:
            cases.append(m)
        else:
            others.append(m)

    final = []

    # 1️⃣ 제3조 계열 법률은 무조건 포함
    for m in laws:
        if m["metadata"].get("article", "").startswith("제3조"):
            final.append(m)
            break

    # 2️⃣ 법률 최소 개수 채우기
    for m in laws:
        if m in final:
            continue
        final.append(m)
        if len(final) >= min_law:
            break

    # 3️⃣ 판례 최소 개수 채우기
    for m in cases:
        final.append(m)
        if len(final) >= min_law + min_case:
            break

    # 4️⃣ 나머지 점수 순으로 채우기
    for m in ranked:
        if m in final:
            continue
        final.append(m)
        if len(final) >= top_k:
            break

    return final

# =========================
# 실행 테스트
# =========================
if __name__ == "__main__":
    query = "확정일자가 없으면 대항력이 없나요?"

    results = retrieve_final(
        query,
        top_k=6,
        min_law=2,
        min_case=1
    )

    for i, m in enumerate(results, 1):
        md = m["metadata"]
        print(f"\n[{i}] score: {m['score']:.4f}")
        print(f"priority: {md.get('priority')}")
        print(f"법종류: {md.get('law_type')}")
        print(f"조문/사례: {md.get('article')} {md.get('chunk_id')}")
        print(f"파일: {md.get('source')}")
        print("-" * 60)
        print(md.get("text", "")[:400])


[1] score: 0.2682
priority: 1
법종류: 주택임대차보호법
조문/사례: 제3조의3 항②
파일: 주택임대차보호법(법률)(제21065호)(20260102).docx
------------------------------------------------------------
임차인이 대항력이나 우선변제권을 갖추고 「민법」

[2] score: 0.2581
priority: 1
법종류: 주택임대차보호법
조문/사례: 제74조 본문
파일: 주택임대차보호법(법률)(제21065호)(20260102).docx
------------------------------------------------------------
제1호부터 제6호까지의 사항 외에 다음 각 호의 사항을 적어야 하며, 이를 증명할 수 있는 서면(임대차의 목적이 주택의 일부분인 경우에는 해당 부분의 도면을 포함한다)을 첨부하여야 한다.<개정 2011. 4. 12., 2020. 2. 4.>
1. 주민등록을 마친 날
2. 임차주택을 점유(占有)한 날
3. 임대차계약증서상의 확정일자를 받은 날
[전문개정 2008. 3. 21.]

[3] score: 0.3124
priority: 50
법종류: 규칙
조문/사례: 제3조 본문
파일: 주택임대차계약증서상의 확정일자 부여 및 임대차 정보제공에 관한 규칙(법무부령)(제01022호)(20220207).docx
------------------------------------------------------------
(확정일자 부여 시 확인사항) 확정일자부여기관은 계약증서에 확정일자를 부여하기 전에 다음 각 호의 사항을 확인하여야 한다. <개정 2016. 5. 23.>
1. 임대인ㆍ임차인의 인적사항, 임대차목적물, 임대차기간, 차임ㆍ보증금 등이 적혀 있는 완성된 문서일 것
2. 계약당사자(대리인에 의하여 계약이 체결된 경우에는 그 대리인을 말한다. 이하 같다)의 서명 또는 기명날인이 있을 것
3. 글자가 연결되어야 할 부분에 빈 공간이 있는 경우에는 계약

In [28]:
import pandas as pd

df = pd.read_csv("data/processed/all_chunks.csv")

df["law_type"].value_counts()

law_type
사례/판례/피해상담    91
주택임대차보호법      57
시행령           39
민법            38
시행규칙/대법원규칙    23
Name: count, dtype: int64

In [29]:
# =========================================
# Pinecone 검색 결과 priority 기반 rerank
# + LLM용 context 생성 (단일 셀)
# =========================================

def rerank_by_priority(matches, max_docs=3):
    """
    Pinecone 검색 결과(matches)를
    1) priority (낮을수록 중요)
    2) score (높을수록 유사)
    기준으로 재정렬 후 상위 max_docs 반환
    """

    def sort_key(m):
        md = m.get("metadata", {})

        # priority가 없으면 최하위로 처리
        priority = md.get("priority", 999)

        # Pinecone 유사도 점수 (없으면 0)
        score = m.get("score", 0)

        # priority 우선, score는 보조
        return (priority, -score)

    # rerank 수행
    sorted_matches = sorted(matches, key=sort_key)

    return sorted_matches[:max_docs]


def build_context_with_priority_rerank(matches, max_docs=3):
    """
    rerank된 검색 결과를
    LLM에 바로 넣을 수 있는 context 문자열로 변환
    """

    # priority 기준 재정렬
    reranked_matches = rerank_by_priority(matches, max_docs)

    chunks = []
    for m in reranked_matches:
        md = m.get("metadata", {})

        chunk = f"""
[출처]
- 법종류: {md.get("law_type")}
- 조문: {md.get("article")} {md.get("chunk_id")}
- 파일: {md.get("source")}

[내용]
{md.get("text")}
"""
        chunks.append(chunk.strip())

    # chunk 사이 구분선
    return "\n\n---\n\n".join(chunks)


# =========================================
# 사용 예시
# =========================================

# 1. Pinecone 검색
res = index.query(
    vector=qvec,
    top_k=10,              # 넉넉히 가져온 뒤
    include_metadata=True  # rerank를 위해 metadata 필수
)

# 2. priority rerank + context 생성
context = build_context_with_priority_rerank(
    res["matches"],
    max_docs=3
)

# context를 LLM 프롬프트에 그대로 사용하면 됨
context


'[출처]\n- 법종류: 주택임대차보호법\n- 조문: 제74조 본문\n- 파일: 주택임대차보호법(법률)(제21065호)(20260102).docx\n\n[내용]\n제1호부터 제6호까지의 사항 외에 다음 각 호의 사항을 적어야 하며, 이를 증명할 수 있는 서면(임대차의 목적이 주택의 일부분인 경우에는 해당 부분의 도면을 포함한다)을 첨부하여야 한다.<개정 2011. 4. 12., 2020. 2. 4.>\n1. 주민등록을 마친 날\n2. 임차주택을 점유(占有)한 날\n3. 임대차계약증서상의 확정일자를 받은 날\n[전문개정 2008. 3. 21.]\n\n---\n\n[출처]\n- 법종류: 규칙\n- 조문: 제9조 항①\n- 파일: 주택임대차계약증서의 확정일자 부여 및 정보제공에 관한 규칙(대법원규칙)(제02986호)(20210610).docx\n\n[내용]\n확정일자부여기관은 전산정보처리조직을 이용하여 다음 각 호의 확정일자정보를 기록한 확정일자부(이하 “전자확정일자부”라 한다)를 작성하여야 한다.\n1. 확정일자번호\n2. 확정일자 부여일\n3. 임대인ㆍ임차인의 인적 정보\n가. 자연인인 경우\n성명, 주민등록번호(외국인인 경우 외국인등록번호), 주소\n나. 법인ㆍ법인 아닌 단체인 경우\n법인명ㆍ단체명, 법인등록번호ㆍ부동산등기용등록번호, 본점 소재지ㆍ주사무소 소재지\n4. 주택의 소재지\n5. 임대차목적물\n6. 임대차기간\n7. 차임ㆍ보증금\n8. 신청인의 성명, 생년월일\n\n---\n\n[출처]\n- 법종류: 규칙\n- 조문: 제9조 항②\n- 파일: 주택임대차계약증서의 확정일자 부여 및 정보제공에 관한 규칙(대법원규칙)(제02986호)(20210610).docx\n\n[내용]\n제1항의 확정일자정보는 20년간 보존한다.'